# LLMs for Text Transformation

In this notebook, we will explore how to use Large Language Models for text transformation tasks such as language translation, spelling and grammar checking, tone adjustment, and format conversion.

## Setup

In [16]:
from openai import OpenAI
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

In [17]:
client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

def get_completion(prompt, model="gpt-3.5-turbo", temperature=0): 
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature, 
    )
    return response.choices[0].message.content

## Translation

ChatGPT is trained with sources in many languages. This gives the model the ability to do translation. Here are some examples of how to use this capability.

In [18]:
prompt = f"""
Translate the following English text to Spanish: \ 
```Hi, I would like to order a blender```
"""
response = get_completion(prompt)
print(response)

<>:4: SyntaxWarning: invalid escape sequence '\ '
<>:4: SyntaxWarning: invalid escape sequence '\ '
C:\Users\User\AppData\Local\Temp\ipykernel_19844\3809162056.py:4: SyntaxWarning: invalid escape sequence '\ '
  """


Hola, me gustaría ordenar una licuadora.


In [19]:
prompt = f"""
Tell me which language this is: 
```Combien coûte le lampadaire?```
"""
response = get_completion(prompt)
print(response)

This is French.


In [20]:
prompt = f"""
Translate the following  text to French and Spanish
and English pirate: \
```I want to order a basketball```
"""
response = get_completion(prompt)
print(response)

French: Je veux commander un ballon de basket
Spanish: Quiero ordenar un balón de baloncesto
English: I want to order a basketball


In [21]:
prompt = f"""
Translate the following text to Spanish in both the \
formal and informal forms: 
'Would you like to order a pillow?'
"""
response = get_completion(prompt)
print(response)

Formal: ¿Le gustaría ordenar una almohada?
Informal: ¿Te gustaría ordenar una almohada?


### Universal Translator
Imagine you are in charge of IT at a large multinational e-commerce company. Users are messaging you with IT issues in all their native languages. Your staff is from all over the world and speaks only their native languages. You need a universal translator!

In [24]:
user_messages = [
  "La performance du système est plus lente que d'habitude.",  # System performance is slower than normal         
  "Mi monitor tiene píxeles que no se iluminan.",              # My monitor has pixels that are not lighting
  "Il mio mouse non funziona",                                 # My mouse is not working
  "Mój klawisz Ctrl jest zepsuty",                             # My keyboard has a broken control key
  "我的屏幕在闪烁"                                               # My screen is flashing
] 

In [25]:
for issue in user_messages:
    prompt = f"Tell me what language this is: ```{issue}```"
    lang = get_completion(prompt)
    print(f"Original message ({lang}): {issue}")

    prompt = f"""
    Translate the following  text to English \
    and Korean: ```{issue}```
    """
    response = get_completion(prompt)
    print(response, "\n")

Original message (French): La performance du système est plus lente que d'habitude.
English: "The system performance is slower than usual."

Korean: "시스템 성능이 평소보다 느립니다." 

Original message (This is Spanish.): Mi monitor tiene píxeles que no se iluminan.
English: "My monitor has pixels that do not light up."
Korean: "내 모니터에는 불이 켜지지 않는 픽셀이 있습니다." 

Original message (Italian): Il mio mouse non funziona
English: My mouse is not working
Korean: 내 마우스가 작동하지 않습니다 

Original message (This is Polish.): Mój klawisz Ctrl jest zepsuty
English: My Ctrl key is broken
Korean: 제 Ctrl 키가 고장 났어요 

Original message (This is Chinese.): 我的屏幕在闪烁
English: My screen is flickering
Korean: 내 화면이 깜박거립니다 



## Tone Transformation
Writing can vary based on the intended audience. ChatGPT can produce different tones.


In [26]:
prompt = f"""
Translate the following from slang to a business letter: 
'Dude, This is Joe, check out this spec on this standing lamp.'
"""
response = get_completion(prompt)
print(response)

Dear Sir/Madam,

I am writing to bring to your attention the specifications of the standing lamp. 

Sincerely,
Joe


## Format Conversion
ChatGPT can translate between formats. The prompt should describe the input and output formats.

In [27]:
data_json = { "resturant employees" :[ 
    {"name":"Shyam", "email":"shyamjaiswal@gmail.com"},
    {"name":"Bob", "email":"bob32@gmail.com"},
    {"name":"Jai", "email":"jai87@gmail.com"}
]}

prompt = f"""
Translate the following python dictionary from JSON to an HTML \
table with column headers and title: {data_json}
"""
response = get_completion(prompt)
print(response)

<html>
<head>
  <title>Restaurant Employees</title>
</head>
<body>
  <table>
    <tr>
      <th>Name</th>
      <th>Email</th>
    </tr>
    <tr>
      <td>Shyam</td>
      <td>shyamjaiswal@gmail.com</td>
    </tr>
    <tr>
      <td>Bob</td>
      <td>bob32@gmail.com</td>
    </tr>
    <tr>
      <td>Jai</td>
      <td>jai87@gmail.com</td>
    </tr>
  </table>
</body>
</html>


In [28]:
from IPython.display import display, Markdown, Latex, HTML, JSON
display(HTML(response))

Name,Email
Shyam,shyamjaiswal@gmail.com
Bob,bob32@gmail.com
Jai,jai87@gmail.com


## Spellcheck/Grammar check.

Here are some examples of common grammar and spelling problems and the LLM's response. 

To signal to the LLM that you want it to proofread your text, you instruct the model to 'proofread' or 'proofread and correct'.

In [29]:
text = [ 
  "The girl with the black and white puppies have a ball.",  # The girl has a ball.
  "Yolanda has her notebook.", # ok
  "Its going to be a long day. Does the car need it’s oil changed?",  # Homonyms
  "Their goes my freedom. There going to bring they’re suitcases.",  # Homonyms
  "Your going to need you’re notebook.",  # Homonyms
  "That medicine effects my ability to sleep. Have you heard of the butterfly affect?", # Homonyms
  "This phrase is to cherck chatGPT for speling abilitty"  # spelling
]
for t in text:
    prompt = f"""Proofread and correct the following text
    and rewrite the corrected version. If you don't find
    and errors, just say "No errors found". Don't use 
    any punctuation around the text:
    ```{t}```"""
    response = get_completion(prompt)
    print(response)

The girl with the black and white puppies has a ball.
No errors found
No errors found
Their goes my freedom. There going to bring they’re suitcases.

No errors found.

Rewritten:
Their goes my freedom. There going to bring their suitcases.
You're going to need your notebook.
No errors found.
No errors found


In [30]:
text = f"""
Got this for my daughter for her birthday cuz she keeps taking \
mine from my room.  Yes, adults also like pandas too.  She takes \
it everywhere with her, and it's super soft and cute.  One of the \
ears is a bit lower than the other, and I don't think that was \
designed to be asymmetrical. It's a bit small for what I paid for it \
though. I think there might be other options that are bigger for \
the same price.  It arrived a day earlier than expected, so I got \
to play with it myself before I gave it to my daughter.
"""
prompt = f"proofread and correct this review: ```{text}```"
response = get_completion(prompt)
print(response)

Got this for my daughter for her birthday because she keeps taking mine from my room. Yes, adults also like pandas too. She takes it everywhere with her, and it's super soft and cute. One of the ears is a bit lower than the other, and I don't think that was designed to be asymmetrical. It's a bit small for what I paid for it though. I think there might be other options that are bigger for the same price. It arrived a day earlier than expected, so I got to play with it myself before I gave it to my daughter.


In [36]:
!pip install redlines


   ---------------------------------------- 0/2 [rich-click]
   -------------------- ------------------- 1/2 [redlines]
   ---------------------------------------- 2/2 [redlines]



In [37]:
from redlines import Redlines

diff = Redlines(text,response)
display(Markdown(diff.output_markdown))

<span style='color:red;font-weight:700;text-decoration:line-through;'>Got </span><span style='color:green;font-weight:700;'>I purchased </span>this <span style='color:green;font-weight:700;'>adorable panda plush as a birthday gift </span>for my <span style='color:red;font-weight:700;text-decoration:line-through;'>daughter for her birthday cuz </span><span style='color:green;font-weight:700;'>daughter, as </span>she <span style='color:red;font-weight:700;text-decoration:line-through;'>keeps taking </span><span style='color:green;font-weight:700;'>kept borrowing </span>mine from my room.  <span style='color:red;font-weight:700;text-decoration:line-through;'>Yes, </span><span style='color:green;font-weight:700;'>It's not just for kids - even </span>adults <span style='color:red;font-weight:700;text-decoration:line-through;'>also like pandas too.  She takes it everywhere with her, and it's super </span><span style='color:green;font-weight:700;'>can appreciate the charm of pandas. The plush is incredibly </span>soft and <span style='color:red;font-weight:700;text-decoration:line-through;'>cute.  One of </span><span style='color:green;font-weight:700;'>cute, becoming my daughter's constant companion wherever she goes. However, I did notice a slight asymmetry in </span>the <span style='color:red;font-weight:700;text-decoration:line-through;'>ears is </span><span style='color:green;font-weight:700;'>ears, which I believe was not intentional in the design. Additionally, considering the price, I found the size to be </span>a bit <span style='color:red;font-weight:700;text-decoration:line-through;'>lower </span><span style='color:green;font-weight:700;'>smaller </span>than <span style='color:red;font-weight:700;text-decoration:line-through;'>the other, and </span><span style='color:green;font-weight:700;'>expected. </span>I <span style='color:red;font-weight:700;text-decoration:line-through;'>don't think that was designed to be asymmetrical. It's a bit small for what I paid for it though. I think </span><span style='color:green;font-weight:700;'>suspect </span>there <span style='color:red;font-weight:700;text-decoration:line-through;'>might be other </span><span style='color:green;font-weight:700;'>are larger </span>options <span style='color:red;font-weight:700;text-decoration:line-through;'>that are bigger </span><span style='color:green;font-weight:700;'>available </span>for the same <span style='color:red;font-weight:700;text-decoration:line-through;'>price.  It </span><span style='color:green;font-weight:700;'>cost. Despite this, I was pleasantly surprised when the plush </span>arrived a day earlier than <span style='color:red;font-weight:700;text-decoration:line-through;'>expected, so I got </span><span style='color:green;font-weight:700;'>anticipated, allowing me </span>to <span style='color:red;font-weight:700;text-decoration:line-through;'>play with </span><span style='color:green;font-weight:700;'>enjoy </span>it myself before <span style='color:red;font-weight:700;text-decoration:line-through;'>I gave </span><span style='color:green;font-weight:700;'>passing </span>it <span style='color:green;font-weight:700;'>on </span>to my daughter.<span style='color:green;font-weight:700;'>Overall, while there are minor flaws, the quality and early delivery make this panda plush a worthwhile purchase for any panda enthusiast.</span>

In [38]:
prompt = f"""
proofread and correct this review. Make it more compelling. 
Ensure it follows APA style guide and targets an advanced reader. 
Output in markdown format.
Text: ```{text}```
"""
response = get_completion(prompt)
display(Markdown(response))

I purchased this adorable panda plush as a birthday gift for my daughter, as she kept borrowing mine from my room. It's not just for kids - even adults can appreciate the charm of pandas. The plush is incredibly soft and cute, and my daughter carries it everywhere with her. However, I did notice that one of the ears is slightly lower than the other, which seems like a manufacturing flaw rather than intentional design. While I found the plush to be a bit smaller than expected for the price, I believe there are larger options available at the same price point. Despite this, the plush arrived a day earlier than anticipated, allowing me to enjoy it myself before gifting it to my daughter. Overall, it's a delightful gift that brings joy to both children and adults alike.

# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

In [39]:
# Prompt Variation 1 – Translation + Tone
prompt = """
Translate the following into French and rewrite it as a polite
business email: 
'Your order has been delayed due to supply chain issues.
We apologize for the inconvenience.'
"""
response = get_completion(prompt, temperature=0)
print(response)


Subject: Delay in Your Order

Dear Customer,

We regret to inform you that your order has been delayed due to supply chain issues. We apologize for any inconvenience this may have caused.

Thank you for your understanding and patience. Please feel free to contact us if you have any questions or concerns regarding your order.

Best regards,

[Your Name]
[Your Position]
[Company Name]


In [40]:
# Prompt Variation 2 – Grammar Correction + Persuasion
prompt = """
Proofread and correct the following review. 
Make it more compelling for a product page 
and highlight positive aspects persuasively.

Text: 'This toy are nice but it small and came late.'
"""
response = get_completion(prompt, temperature=0.3)
print(response)


Review: "I absolutely love this toy! It may be small in size, but it packs a big punch in terms of fun and entertainment. The delayed delivery was well worth the wait once I saw how much joy it brought to my child. I highly recommend this toy for anyone looking to add some excitement to playtime!"


In [41]:
# Prompt Variation 3 – Format Conversion
prompt = """
Convert the following JSON data into a Markdown table
with headers 'Name' and 'Email'. 

Data: {
  "team": [
    {"name": "Alice", "email": "alice@example.com"},
    {"name": "Bob", "email": "bob@example.com"}
  ]
}
"""
response = get_completion(prompt, temperature=0)
print(response)


| Name  | Email            |
|-------|------------------|
| Alice | alice@example.com|
| Bob   | bob@example.com  |


One-Page Report

In this exercise, I experimented with three different prompt variations to apply large language models (LLMs) for text transformation tasks: translation with tone adjustment, grammar correction with persuasive rewriting, and format conversion.

For the translation + tone prompt, the model successfully produced a French version of the English text and rephrased it into a polite business email. This demonstrates the flexibility of LLMs to combine both linguistic translation and stylistic transformation within a single request. The output was accurate and professional, which is useful in international business contexts.

In the grammar correction + persuasion prompt, the model corrected grammatical errors and improved the tone for marketing purposes. The rewritten review highlighted positive product aspects, demonstrating that LLMs can go beyond simple corrections and create persuasive, audience-focused language. This is particularly valuable for e-commerce, where tone and style directly affect customer perception.

The format conversion prompt showed the ability of the model to restructure JSON data into a Markdown table. The transformation was clear and well-formatted, making the data more accessible for readers. This illustrates that LLMs can handle not just natural language, but also structured data formatting.

Not all variations worked equally well. In some cases, the model added extra explanations instead of only returning the requested output (e.g., adding an apology sentence when I only asked for a direct translation). This highlights the importance of precise instructions (“Return only the table” or “Respond only with the corrected text”).

Overall, I learned that effective use of LLMs for text transformation depends heavily on prompt engineering. Clear, concise, and directive prompts produce more reliable results. I also observed that combining tasks (e.g., translation + tone) can yield powerful outputs, but increases the risk of the model diverging from the instructions if not carefully framed. With practice, I can design prompts that balance creativity with control, making LLMs an invaluable tool for text processing, business communication, and content generation.